In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import elasticsearch.exceptions

In [2]:
# Check connection!

import requests
from requests.auth import HTTPBasicAuth

dbUrl = "https://43e94a6daea349b88b7fe5df6a59e496.ap-southeast-2.aws.found.io:9243/"
res = requests.get(dbUrl, auth=HTTPBasicAuth("elastic", "GL0nVKx2szcQtBKaJVDxAOmf"))

print(res.status_code)
print(res.ok)

200
True


In [3]:
# Elastic search object
es = Elasticsearch([dbUrl], 
                   http_auth=('elastic', 'GL0nVKx2szcQtBKaJVDxAOmf'))

In [ ]:
# Manual DB update
def gendata():
    refids = [1,2,3]
    topics = ["a", "b", "c"]
    searchString = ['foo', 'bar', 'baz']
    answers = ['foo-found', 'bar-found', 'baz-found']
    i=0
    
    while i < len(refids):
        yield {
            "_index": "salesforce_kb",
            "_type": "_doc",
            "refID": refids[i],
            "topic": topics[i],
            "searchString": searchString[i],
            "refurl": "",
            "lastModified": "",
            "answer": answers[i]
        }
        i+=1

In [ ]:
# DB store/update
try:
    bulk(es, gendata(), 
         #index='TestChatmate', 
         #doc_type='clue', 
         raise_on_error=True)
except elasticsearch.exceptions.ConnectionError:
    # TODO: Handle the error appropriately for your situation
    print("Couldn't connect to DB")
else:
    print("record added to DB!")

In [4]:
# Chatmate interface
def chatmate():
    quit = False
    print("Chatmate: What can I help you with, mate?")
    while quit == False:
        user_input = str(input('Me: '))
        ##an optional quit command
        if user_input.casefold() == 'quit' or user_input.casefold() == 'bye':
            print("Chatmate: Bye!")
            quit = True
        else:
            response = es.search(index='salesforce_kb', doc_type='_doc', body={ "query": {
                "match": {
                    "searchString":{
                        "query":user_input
                    }
                }
            }})
            try:
                ans = response['hits']['hits'][0]['_source']['answer']
                print("Chatmate : {}".format(ans))
            except IndexError:
                print("Chatmate: Can't do that!")

In [6]:
chatmate()

Chatmate: What can I help you with, mate?
Me: Hi!
Chatmate: Can't do that!
Me: what is your name?
Chatmate : My name is Zain
Me: sydney
Chatmate: Can't do that!
Me: quit
Chatmate: Bye!


In [10]:
import csv,json
csvFilePath = 'csv_file_name.csv'
jsonFilePath = 'json_file_name.json'
convertedData = {}

with open(csvFilePath) as csvFile:
    csvReader = csv.DictReader(csvFile)
    for rows in csvReader:
        id = rows['index']
        convertedData[id] = rows

with open(jsonFilePath, 'w') as jsonFile:
    jsonFile.write(json.dumps(convertedData,indent=4))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 153: invalid continuation byte